In [ ]:
import numpy as np
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pickle

with open("opt_guess_xf_0.1_yf_0.98.pickle", "wb") as file:
    pickle.dump(opt_guess, file, protocol=pickle.HIGHEST_PROTOCOL)

# with open('curvature_alpha30_N150_xf0.1_yf_0.98/opt_guess_xf_0.1_yf_0.98.pickle', 'rb') as file:
#     opt_guess = pickle.load(file)

In [ ]:
def construct_initial_guess(x_opt: np.ndarray, u_opt: np.ndarray, N: int) -> list:
    """
    Construct an initial guess using a given solution.
    """
    w0 = []
    for i in range(N + 1):
        state = list(x_opt[:, i])
        if i == 0:

            w0.append(state)
            u = [u_opt[0, i]]
            w0.append(u)
        elif i == N:
            for j in range(4):
                w0.append(state)
        elif i > 0 and i < N:
            for j in range(4):
                w0.append(state)
            u = [u_opt[0, i]]
            w0.append(u)
    return w0


# This code takes a solution from a simulation with N control steps and
# upsamples it so it can be used as an intial guess for a simulation with 2*N
# control steps
def upsampled_initial_guess(w0: list, x_opt: np.ndarray, u_opt: np.ndarray) -> list:
    """
    Creates an initial guess for a problem with 2*N control intervals.

    Starts with a guess for a solution to the problem with N intervals.

    Parameters
    ----------
    w0 : list
        Initial control parameters.

    Returns
    -------
    w_new : list
        Initial guess for problem with 2*N control intervals.
    """
    w_new = []
    w_new.append(w0[0])
    for i in range(N):
        for k in range(2):
            w_new.append([u_opt[0, i]])
            for j in range(4):
                state = list(x_opt[:, i + 1])
                w_new.append(state)
    return w_new


def construct_initial_guess_upsampled(
    x_opt: np.ndarray, u_opt: np.ndarray, N: int, new_N: int
) -> list:
    """
    Construct an initial guess using a given solution.
    """
    w0 = []
    print(new_N)
    for i in range(new_N + 1):
        if i > N - 1:
            k = -1
        else:
            k = i
        state = list(x_opt[:, k])
        if i == 0:
            w0.append(state)
            u = [u_opt[0, i]]
            w0.append(u)
        elif i == new_N:
            for j in range(4):
                w0.append(state)
        elif i > 0 and i < new_N:
            for j in range(4):
                w0.append(state)
            u = [u_opt[0, k]]
            w0.append(u)
    return w0

In [ ]:
# Time horizon. For me this is the length of the snake
T = 1
init_guess = opt_guess
# Control discretization
N = 150  # number of control intervals
h = T / N

# Degree of interpolating polynomial
d = 3

# Get collocation points
tau_root = np.append(0, ca.collocation_points(d, "legendre"))

# Coefficients of the collocation equation
C = np.zeros((d + 1, d + 1))

# Coefficients of the continuity equation
D = np.zeros(d + 1)

# Coefficients of the quadrature function
B = np.zeros(d + 1)

# Construct polynomial basis
for j in range(d + 1):
    # Construct Lagrange polynomials to get the polynomial basis at the collocation point
    p = np.poly1d([1])
    for r in range(d + 1):
        if r != j:
            p *= np.poly1d([1, -tau_root[r]]) / (tau_root[j] - tau_root[r])

    # Evaluate the polynomial at the final time to get the coefficients of the continuity equation
    D[j] = p(1.0)

    # Evaluate the time derivative of the polynomial at all collocation points to get the coefficients of the continuity equation
    pder = np.polyder(p)
    for r in range(d + 1):
        C[j, r] = pder(tau_root[r])

    # Evaluate the integral of the polynomial to get the coefficients of the quadrature function
    pint = np.polyint(p)
    B[j] = pint(1.0)


# Declare model variables
x0 = ca.SX.sym("x0")
x1 = ca.SX.sym("x1")
x2 = ca.SX.sym("x2")
x3 = ca.SX.sym("x3")
x4 = ca.SX.sym("x4")
x5 = ca.SX.sym("x5")
x = ca.vertcat(x0, x1, x2, x3, x4, x5)
u = ca.SX.sym("u")  # the control for me is the active moment

# model parameter rho*g/B
alpha = 50

# Model equations
xdot = ca.vertcat(1, ca.cos(x3), ca.sin(x3), x4, alpha * (x0 - 1) * ca.cos(x3) - u, u)

# Objective term. For now it's just the size of the cost
L = x4**2

# Continuous time dynamics
f = ca.Function("f", [x, u], [xdot, L], ["x", "u"], ["xdot", "L"])


# Start with an empty NLP
w = []
w0 = []
lbw = []
ubw = []
J = 0
g = []
lbg = []
ubg = []

# For plotting x and u given w
x_plot = []
u_plot = []

# control bounds
u_lb = -10
u_ub = 10

# state dimension
state_dim = 6

# state bounds (s, x, y, theta, dtheta_ds, a)
state_lb = [0, -np.inf, -np.inf, -np.inf, -10, -np.inf]
state_ub = [T, np.inf, np.inf, np.inf, 10, np.inf]

# initial state
initial_state = [0, 0, 0, 0.5, 0.5, -10]
# initial_state = opt_guess

# "Lift" initial conditions
Xk = ca.MX.sym("X0", state_dim)
w.append(Xk)
lbw.append([0, 0, 0, -0.75, -10, -10])  # allowing variation in initial conditions
ubw.append([0, 0, 0, 0.75, 10, 10])
w0.append(initial_state)
x_plot.append(Xk)


# Formulate the NLP
for k in range(N):
    # New NLP variable for the control
    Uk = ca.MX.sym("U_" + str(k))
    w.append(Uk)
    lbw.append([u_lb])
    ubw.append([u_ub])
    w0.append([u_lb])  # initial control
    u_plot.append(Uk)

    # State at collocation points
    Xc = []
    for j in range(d):
        Xkj = ca.MX.sym("X_" + str(k) + "_" + str(j), state_dim)
        Xc.append(Xkj)
        w.append(Xkj)
        lbw.append(state_lb)
        ubw.append(state_ub)
        w0.append(initial_state)

    # Loop over collocation points
    Xk_end = D[0] * Xk
    for j in range(1, d + 1):
        # Expression for the state derivative at the collocation point
        xp = C[0, j] * Xk
        for r in range(d):
            xp = xp + C[r + 1, j] * Xc[r]

        # Append collocation equations
        fj, qj = f(Xc[j - 1], Uk)
        g.append(h * fj - xp)
        lbg.append([0] * state_dim)
        ubg.append([0] * state_dim)

        # Add contribution to the end state
        Xk_end = Xk_end + D[j] * Xc[j - 1]

        # Add contribution to quadrature function
        J = J + B[j] * qj * h

    # New NLP variable for state at end of interval
    Xk = ca.MX.sym("X_" + str(k + 1), state_dim)
    w.append(Xk)
    lbw.append(state_lb)
    ubw.append(state_ub)
    w0.append(initial_state)
    x_plot.append(Xk)

    # Add equality constraint
    g.append(Xk_end - Xk)
    lbg.append([0] * state_dim)
    ubg.append([0] * state_dim)

# Final state
x_final_val = 0.1
y_final_val = 0.98
eq1 = x1 - x_final_val
eq2 = x2 - y_final_val
eq3 = x3 - np.pi / 2
eq4 = x4
eq = ca.vertcat(eq1, eq2, eq3, eq4)

xf_eq = ca.Function("xf_eq", [x], [eq], ["x"], ["eq"])

g.append(xf_eq(Xk))  # Xk is the final state
n_eq = 4  # number of final equations
lbg.append([0] * n_eq)  # allowed deviation
ubg.append([0] * n_eq)

# Concatenate vectors
w = ca.vertcat(*w)
g = ca.vertcat(*g)
x_plot = ca.horzcat(*x_plot)
u_plot = ca.horzcat(*u_plot)
w0 = np.concatenate(w0)
lbw = np.concatenate(lbw)
ubw = np.concatenate(ubw)
lbg = np.concatenate(lbg)
ubg = np.concatenate(ubg)

# Create an NLP solver
prob = {"f": J, "x": w, "g": g}
ipopt_options = {
    "print_time": True,
    "ipopt.max_iter": 100,
}

solver = ca.nlpsol("solver", "ipopt", prob, ipopt_options)


# Function to get x and u trajectories from w
trajectories = ca.Function("trajectories", [w], [x_plot, u_plot], ["w"], ["x", "u"])

# Solve the NLP
# sol = solver(x0=double_guess, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
sol = solver(x0=init_guess, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
x_opt, u_opt = trajectories(sol["x"])
x_opt = x_opt.full()  # to numpy array
u_opt = u_opt.full()  # to numpy array

# Plot the result
tgrid = np.linspace(0, T, N + 1)
plt.figure(1)
plt.clf()
plt.plot(tgrid, x_opt[0], "--")
plt.plot(tgrid, x_opt[1], "-")
plt.plot(tgrid, x_opt[2], "o")
plt.step(tgrid, np.append(np.nan, u_opt[0]), "-.")
plt.xlabel("t")
plt.legend(["s", "x", "y", "u"])
plt.grid()
plt.show()

In [ ]:
plt.plot(-x_opt[1], x_opt[2])
plt.axis("scaled")
plt.xlim([-0.12, 0])
plt.title("Snake shape")
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
fig, ax = plt.subplots(nrows=2, ncols=2, sharex=True)
ax[0, 0].step(tgrid, np.append(np.nan, u_opt[0]))
ax[0, 0].set_title("control")
ax[0, 1].plot(tgrid, np.append(np.nan, u_opt[0]))
ax[0, 1].plot(tgrid, x_opt[4])
ax[0, 1].set_title("Control and curvature")
ax[1, 0].plot(tgrid, x_opt[3])
ax[1, 0].set_title("Snake angle")
ax[1, 1].plot(tgrid, x_opt[5])
ax[1, 1].set_title("Active moment")
fig.tight_layout()
plt.show()

In [ ]:
plt.plot(tgrid, -x_opt[4])
plt.xlabel("s", fontsize=16)
plt.ylabel(r"$\kappa$", fontsize=16)
plt.title("Curvature", fontsize=20)
plt.show()

In [ ]:
opt_guess = sol["x"]
w0 = construct_initial_guess(x_opt, u_opt, N=N)
w0_arr = np.concatenate(w0)
np.save("w0", w0_arr)

double_w = upsampled_initial_guess(w0, x_opt, u_opt)
w_new_arr = np.concatenate(double_w)
np.save("double_w0", w_new_arr)

upsamp = construct_initial_guess_upsampled(x_opt, u_opt, N, new_N=160)
upsamp = np.concatenate(upsamp)

In [ ]:
# double_guess = np.load("double_w0.npy")